In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from natsort import natsorted

In [ ]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a = 5/3

In [ ]:
import xarray as xr
import imageio

def create_movie(field_time_out, monitor_lambdas,name='',type='t',log=False,path="",frames_per_second=1,rem_frames=False, normalize=True, a=5/3):
    frames = []

    max_values = field_time_out.max(axis=(0,1,2)) if normalize else 1

    field_time_out = field_time_out/max_values
    field_time_out['x'] = field_time_out['x']/a
    field_time_out['y'] = field_time_out['y']/a
    field_time_out['z'] = field_time_out['z']/a


    if log:
        field_log = np.log10((field_time_out))
        folder_pics = "logPics"
        
    else:
        field_log = ((field_time_out))
        folder_pics = "linPics"

    if not os.path.exists(f'{path}/{folder_pics}'):
            os.makedirs(f'{path}/{folder_pics}')
            print(f"Folder {path}/{folder_pics} created successfully.")

    for i, time in enumerate(field_time_out.t):
        if os.path.isfile(f'{path}/{folder_pics}/frame_{i}.png'):
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
            continue
        try:
            #fig, ax = plt.subplots(figsize=(14/2, 18/2))
            fig, ax = plt.subplots(figsize=(12, 6))
            if type=="t":
                pcolormesh = (field_log).isel(t=i).squeeze().plot.pcolormesh(ax=ax,cmap="jet")
            else:
                pcolormesh = (field_log).isel(f=i).squeeze().plot.pcolormesh(ax=ax,cmap="jet")

            ax.set_aspect('auto', adjustable='box')
            try:
                plt.title(f'Time: {str(np.array(field_time_out['t'][()][i])*1e12)} ps')
            except:
                plt.title(f'$\\nu$: {(1/np.array(td.C_0/field_time_out['f'][()][i])):.4g}')


            # Save the frame
            plt.savefig(f'{path}/{folder_pics}/frame_{i}.png')
            plt.close(fig)
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
        except:
            break
        

    name_movie = f'{path}/{name}.mp4' if name else f'output/anderson/d(t) analysis/Diameter d(t) at output of the structure Range - {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}.mp4'
    with imageio.get_writer(name_movie, fps=frames_per_second) as writer:
        for frame in frames:
            image = imageio.imread(frame)
            writer.append_data(image)

    # Optionally, remove the individual frames if no longer needed
    if rem_frames:
        for frame in frames:
            os.remove(frame)
    
    return False

In [ ]:
folder_path = r"H:\phd stuff\tidy3d\data\08_16_2024 STL RCP Sample Test Transmission Time\z_incidence"
      

for i,filename in enumerate(natsorted(os.listdir(folder_path))):
    if filename != "RCP  0.08-0.10.txt":
        continue
    print(filename)
    file=os.path.join(folder_path, filename)
    sim_object_result = AM.loadFromFile(key = tidy3dAPI, file_path=file)
    print(sim_object_result.__str__())
    sim_data = sim_object_result.sim_data
    sim_data0 = sim_object_result.sim_data0
    transmission0 = sim_data0['flux1'].flux
    transmission = sim_data['flux1'].flux
    transmission_left0 = sim_data0['flux2'].flux
    transmission_left = sim_data['flux2'].flux
    transmission_normalized_left = transmission_left / transmission_left0
    transmission_normalized = transmission / transmission0
    monitor_lambdas = a/(td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs))
    plt.plot(monitor_lambdas,transmission_normalized, label=f"L=Right")
    plt.plot(monitor_lambdas,transmission_normalized, label=f"L=Left")
    plt.xlabel(r'$\nu$')
    plt.ylabel('Transmission')
    plt.legend()
    plt.grid()
    plt.yscale("log")
    plt.title(filename)
    plt.show()

    #Plot T(t)
    flux_t = sim_data["time_monitorT"].flux
    times = flux_t['t']*1e12
    plt.plot(times,flux_t)
    plt.yscale("log")
    plt.grid()
    plt.show()

    field_time_out = sim_data.get_intensity("time_monitorFieldLateral")
    print(np.shape(field_time_out))
   
    create_movie(field_time_out,monitor_lambdas,name=f"timemonitor {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}",type="t",log=False,path=r"H:\phd stuff\tidy3d\output/3D SHU/T(t) Field Map",frames_per_second=10,normalize=False,a=a)


